<a href="https://colab.research.google.com/github/eric88525/DRCD/blob/master/trainDRCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DRCD

In [1]:
!nvidia-smi

Fri Aug  7 15:49:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 該來用TPU了吧
+ [REF](https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/getting-started.ipynb#scrollTo=H9KYz-Vk4fMa)

In [2]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [3]:
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  57471      0 --:--:-- --:--:-- --:--:-- 57471
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
     |████████████████████████████████| 61kB 2.6MB/s 
Uninstalling torch-1.6.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     


In [3]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

In [4]:
# Creates a random tensor on xla:1 (a Cloud TPU core)
device = xm.xla_device()
t1 = torch.ones(3, 3, device = device)
print(t1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='xla:1')


# Setup

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [1]:
!pip install transformers
!pip install opencc
!pip install pyprind
!pip install zhon

KeyboardInterrupt: ignored

In [3]:
import torch
import torch.nn as nn
from torch import optim
from transformers import *
import pandas as pd
import ast
import copy
import os
import json
from time import strftime,gmtime
from opencc import OpenCC
import pyprind
from sklearn.utils import shuffle
import re
from zhon.hanzi import non_stops,stops
import numpy as np
import random
import argparse
import time
from datetime import datetime, timedelta

print(torch.cuda.is_available())

True


In [4]:
cd ./drive/My Drive/Colab Notebooks/DRCD

/content/drive/My Drive/Colab Notebooks/DRCD


In [ ]:
!ls

# Get Train Dev Test

+ train 8014 26936
+ test 1000 3493
+ dev 1000 3524
  

In [5]:
import json

class DRCDdataset():
  def __init__(self,train_path=None , test_path = None ,dev_path = None):
    
    if train_path != None:
      self.train = self.get_data(train_path)
    if test_path != None:
      self.test = self.get_data(test_path)
    if dev_path != None:
      self.dev = self.get_data(dev_path)

  def get_data(self,path):
    input_file = open(path)
    ds = json.load(input_file)
    datas = []
    for i in range(len(ds['data'])):
      for j in ds['data'][i]['paragraphs']:
        context = j['context']
        question = [q['question'] for q in j['qas']]
        ans = [ a['answers'][0]['text'] for a in j['qas'] ]
        #ans = [a['answers'][0]['answer_start'] for a in j['qas']]
        #anse = [a['answers'][0]['answer_start']+len(a['answers'][0]['text'])-1 for a in j['qas']]
        for k in range(len(question)):
          datas.append([context,question[k],ans[k]])
          #datas.append([context,question[k],ans[k],anse[k]])
    return datas


# Dataset

In [6]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class GetDataset(Dataset):
    def __init__(self,data,model_type,device,language):
      self.data = data
      self.tokenizer = BertTokenizer.from_pretrained(model_type)
      self.device = device
      self.cc = OpenCC('t2s') # tw->china
      self.language = language
    def __getitem__(self,idx):
      paragraph,question,ans = self.data[idx][0] , self.data[idx][1], self.data[idx][2] 
      if self.language == 'china':
        paragraph,question,ans = self.cc.convert(paragraph),self.cc.convert(question),self.cc.convert(ans)
        
      token_tensor = self.tokenizer.encode_plus(question,paragraph,max_length=512,truncation=True,pad_to_max_length=True)
      #print(paragraph[:20],question,ans)
      s_idx = [0]*512
      e_idx = [0]*512
      # 答案tok
      s_tok = self.tokenizer.encode(ans)[1:-1]
      # 找到開頭跟他一樣的
      s_lsit = [i for i, x in enumerate(token_tensor['input_ids']) if x == s_tok[0]]
      
      for s_pos in s_lsit:
        e_pos = s_pos+len(s_tok)
        if e_pos > 511:
          continue
          
        if token_tensor['input_ids'][s_pos:e_pos] == s_tok:
          s_idx[s_pos] = 1
          e_idx[e_pos-1] = 1
          break
      s_tensor = torch.Tensor(s_idx)
      e_tensor = torch.Tensor(e_idx)
      # input_ids / token_type_ids / attention_mask / s_tensor / e_tensor
      return {'input_ids': torch.tensor(token_tensor['input_ids']).to(self.device)
          ,'token_type_ids': torch.tensor( token_tensor['token_type_ids']).to(self.device) 
          ,'attention_mask': torch.tensor( token_tensor['attention_mask']).to(self.device)
          ,'s_tensor': s_tensor.to(self.device)
          ,'e_tensor': e_tensor.to(self.device)}
    def __len__(self):
      return len(self.data)
    

In [7]:

x = DRCDdataset(train_path='./dataset/DRCD_train.json',test_path='./dataset/DRCD_test.json',dev_path='./dataset/DRCD_dev.json')
print(len(x.train),len(x.test),len(x.dev))
device = torch.device('cuda:0')
model_type = 'hfl/chinese-roberta-wwm-ext'
language = 'china'
z = GetDataset(x.dev,model_type,device,language)
tokenizer = BertTokenizer.from_pretrained(model_type)
i = 10

tok = tokenizer.convert_ids_to_tokens(z[i]['input_ids'])
s = z[i]['s_tensor'].tolist()
e = z[i]['e_tensor'].tolist()
tp = z[i]['token_type_ids'].tolist()
at = z[i]['attention_mask'].tolist()
for i in range(len(tok)):
  print(tok[i],s[i],e[i],tp[i],at[i])


26936 3493 3524
[CLS] 0.0 0.0 0 1
负 0.0 0.0 0 1
责 0.0 0.0 0 1
管 0.0 0.0 0 1
理 0.0 0.0 0 1
马 0.0 0.0 0 1
祖 0.0 0.0 0 1
国 0.0 0.0 0 1
家 0.0 0.0 0 1
风 0.0 0.0 0 1
景 0.0 0.0 0 1
区 0.0 0.0 0 1
的 0.0 0.0 0 1
单 0.0 0.0 0 1
位 0.0 0.0 0 1
为 0.0 0.0 0 1
？ 0.0 0.0 0 1
[SEP] 0.0 0.0 0 1
马 0.0 0.0 1 1
祖 0.0 0.0 1 1
列 0.0 0.0 1 1
岛 0.0 0.0 1 1
是 0.0 0.0 1 1
隶 0.0 0.0 1 1
属 0.0 0.0 1 1
中 0.0 0.0 1 1
华 0.0 0.0 1 1
民 0.0 0.0 1 1
国 0.0 0.0 1 1
的 0.0 0.0 1 1
群 0.0 0.0 1 1
岛 0.0 0.0 1 1
， 0.0 0.0 1 1
位 0.0 0.0 1 1
于 0.0 0.0 1 1
台 0.0 0.0 1 1
湾 0.0 0.0 1 1
海 0.0 0.0 1 1
峡 0.0 0.0 1 1
正 0.0 0.0 1 1
北 0.0 0.0 1 1
方 0.0 0.0 1 1
， 0.0 0.0 1 1
面 0.0 0.0 1 1
临 0.0 0.0 1 1
闽 0.0 0.0 1 1
江 0.0 0.0 1 1
口 0.0 0.0 1 1
、 0.0 0.0 1 1
连 0.0 0.0 1 1
江 0.0 0.0 1 1
口 0.0 0.0 1 1
和 0.0 0.0 1 1
罗 0.0 0.0 1 1
源 0.0 0.0 1 1
湾 0.0 0.0 1 1
， 0.0 0.0 1 1
与 0.0 0.0 1 1
中 0.0 0.0 1 1
国 0.0 0.0 1 1
大 0.0 0.0 1 1
陆 0.0 0.0 1 1
仅 0.0 0.0 1 1
一 0.0 0.0 1 1
水 0.0 0.0 1 1
之 0.0 0.0 1 1
隔 0.0 0.0 1 1
， 0.0 0.0 1 1
距 0.0 0.0 1 1
中 0.0 0.0 

# Model

In [10]:
class bertDRCD(nn.Module):
    def __init__(self,model_type):
        super(bertDRCD,self).__init__()

        config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
        self.bert_model = BertModel.from_pretrained(model_type,config = config)
  
        self.s_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) 
        self.e_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) 

        #self.start()

    def start(self):
      self.decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,1)
      )     
      nn.init.xavier_uniform_(self.s_decoder[0].weight)
      nn.init.constant_(self.s_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.s_decoder[3].weight)
      nn.init.constant_(self.s_decoder[3].bias, 0)
      nn.init.xavier_uniform_(self.e_decoder[0].weight)
      nn.init.constant_(self.e_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.e_decoder[3].weight)
      nn.init.constant_(self.e_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None): 
      # hidden (batch,seq_len,hidden)
      hidden = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[0]
      s = torch.sum(hidden*self.s_vector,-1) # s now is (batch,seq_len)
      M = token_type_ids.clone().float().to(hidden.device).detach()
      M[M != 1] = float('-inf')
      s = s+M 
      #s = torch.softmax(s,dim=-1)
      e = torch.sum(hidden*self.e_vector,-1)
      e = e + M
      #e = torch.softmax(e,dim=-1)
      return s,e

In [ ]:
"""
device = torch.device('cpu')
device = torch.device('cuda:0')

model_type = 'bert-base-chinese'
model = bertDRCD(model_type).to(device)
ds = DRCDdataset(train_path='./dataset/DRCD_train.json',test_path='./dataset/DRCD_test.json',dev_path='./dataset/DRCD_dev.json')
dataset = GetDataset(ds.dev,model_type,device)
trainLoader = DataLoader(dataset, batch_size=8  ,shuffle=True)
criterion = nn.BCELoss()

for i , batch in enumerate(trainLoader):
  #print(batch)
  inp_ids,tok_id,att_m ,slabel,elabel = batch['input_ids'],batch['token_type_ids'],batch['attention_mask'],batch['s_tensor'],batch['e_tensor']
  s,e = model(input_ids=inp_ids,attention_mask=att_m,token_type_ids=tok_id)
  print(s)
  print(e)
  loss = (criterion(s,slabel) + criterion(e,elabel)) / 2
  print(loss)
  
  break
"""

# Test

In [11]:
def test(model,ds,args,tp):
  if tp == 'test':
    print(f'Testing...{len(dataset)}')
    dataset = GetDataset(ds.test,args.model_type,args.device,args.language)
    
  elif tp == 'dev':   
    dataset = GetDataset(ds.dev,args.model_type,args.device,args.language)
    print(f'Dev...{len(dataset)}')
  loader = DataLoader(dataset,batch_size=args.batch_size,shuffle=True)
  model.eval()
  loss,i = 0,0
  criterion = nn.BCELoss()
  with torch.no_grad():
    for batch in loader:
      i = i+1
      inp_ids,tok_id,att_m ,slabel,elabel = batch['input_ids'],batch['token_type_ids'],batch['attention_mask'],batch['s_tensor'],batch['e_tensor']
      s,e = model(input_ids=inp_ids,attention_mask=att_m,token_type_ids=tok_id)
      batch_loss = (criterion(s,slabel) + criterion(e,elabel)) / 2
      loss += batch_loss
  print(f'Result: LOSS: {loss} AVG: {loss/i} ')
  return loss/i

# Train
+ train(ds,w_d,lr_rate,device,model_type,epoches):
+ data(還沒封裝) , weight_d , learningrate , device , model_type , epoches

In [26]:
def train(ds,args):
  dataset = GetDataset(ds.train,args.model_type,args.device,args.language)
  trainLoader = DataLoader(dataset,batch_size=args.batch_size,shuffle=True)
  model = bertDRCD(args.model_type).to(args.device)
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = AdamW(parameters, lr=args.learning_rate, weight_decay=args.weight_decay)
  criterion = nn.BCELoss()
  model.train()
  minloss = 100000
  for ei in range(args.epoch):
    loss,i,check_loss = 0,0,0
    for batch in trainLoader:
      i+=1
      inp_ids,tok_id,att_m ,slabel,elabel = batch['input_ids'],batch['token_type_ids'],batch['attention_mask'],batch['s_tensor'],batch['e_tensor']
      s,e = model(input_ids=inp_ids,attention_mask=att_m,token_type_ids=tok_id)
      #print(f's {s} \n e {e} \n sl {torch.softmax(s,dim=-1)} \n el {torch.softmax(e,dim=-1)} \n ')
      #print(f'SL {slabel} \n EL {elabel}')
      batch_loss = (criterion(torch.softmax(s,dim=-1),slabel) + criterion(torch.softmax(e,dim=-1),elabel)) / 2
      loss += batch_loss
      check_loss += batch_loss
      #print(batch_loss)
      batch_loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      if i % 500==0:
        print(f'500 batch AVGLOSS {check_loss/1000}')
        check_loss = 0
    print(f'Epoches: {ei} Loss {loss} AVG: {loss/i}')

    if loss < minloss:
      minloss = loss
      best_model = copy.deepcopy(model.state_dict())
  return best_model

# Main

In [27]:
parser = argparse.ArgumentParser([])
parser.add_argument('--batch-size', default=8 , type=int)
parser.add_argument('--epoch', default=5, type=int)
parser.add_argument('--learning-rate', default=1e-5, type=float)    
parser.add_argument('--weight-decay', default=0.001, type=float)
parser.add_argument('--model-type', default='hfl/chinese-roberta-wwm-ext' , type=str)  #model_type = 'hfl/chinese-bert-wwm'  'hfl/chinese-roberta-wwm-ext'  'hfl/chinese-roberta-wwm-ext-large'
parser.add_argument('--device', default=torch.device('cuda:0'), type=int)
parser.add_argument('--language', default='china', type=str)
args = parser.parse_args([])
args

Namespace(batch_size=8, device=device(type='cuda', index=0), epoch=5, language='china', learning_rate=1e-05, model_type='hfl/chinese-roberta-wwm-ext', weight_decay=0.001)

In [24]:
ds = DRCDdataset(train_path='./dataset/DRCD_train.json',test_path='./dataset/DRCD_test.json',dev_path='./dataset/DRCD_dev.json')

In [15]:
ds.train[:5]

[['2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2004年8月5日正式投入運營，屬中國交通情況第二繁忙的機場。該機場取代了原先位於市中心的無法滿足日益增長航空需求的舊機場。目前機場有三條飛機跑道，成為國內第三個擁有三跑道的民航機場。比鄰近的香港國際機場第三跑道預計的2023年落成早8年。',
  '廣州的快速公交運輸系統每多久就會有一輛巴士？',
  '10秒鐘'],
 ['2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2004年8月5日正式投入運營，屬中國交通情況第二繁忙的機場。該機場取代了原先位於市中心的無法滿足日益增長航空需求的舊機場。目前機場有三條飛機跑道，成為國內第三個擁有三跑道的民航機場。比鄰近的香港國際機場第三跑道預計的2023年落成早8年。',
  '從哪一天

In [ ]:
mode = 'train'

if mode == 'train': 
  print('Train')
  best_model = train(ds,args)
  if not os.path.exists('saved_models'):
    os.makedirs('saved_models')    
  modelname = 'bertDRCD'+'_'+(datetime.now()+timedelta(hours=8)).strftime("%m%d_%H%M")+'.pt' 
  torch.save(best_model, f'saved_models/{modelname}')
  print(f'Train end, model name is {modelname}.pt')

elif mode == 'test' or mode == 'dev':
  modelname = 'bertDRCD_0807_2222.pt'
  test_model = bertDRCD(args.model_type).to(args.device)
  test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
  test(test_model,ds,args,mode)

#22.53021812438965
#22.579822540283203

Train
500 batch AVGLOSS 0.06845083832740784
500 batch AVGLOSS 0.07170592993497849
500 batch AVGLOSS 0.07431110739707947
500 batch AVGLOSS 0.06905978918075562
500 batch AVGLOSS 0.06781522184610367
500 batch AVGLOSS 0.06675072759389877
Epoches: 0 Loss 466.5035095214844 AVG: 0.13855168223381042
500 batch AVGLOSS 0.06460896879434586
500 batch AVGLOSS 0.06246434897184372
500 batch AVGLOSS 0.06039474904537201
500 batch AVGLOSS 0.05853394418954849
500 batch AVGLOSS 0.057144973427057266
500 batch AVGLOSS 0.056012626737356186
Epoches: 1 Loss 399.83624267578125 AVG: 0.11875148862600327
500 batch AVGLOSS 0.054885122925043106
500 batch AVGLOSS 0.054207347333431244
500 batch AVGLOSS 0.05415158346295357
500 batch AVGLOSS 0.053444404155015945
500 batch AVGLOSS 0.05336707457900047
500 batch AVGLOSS 0.05288129672408104
Epoches: 2 Loss 361.75439453125 AVG: 0.10744116455316544


# Package

In [ ]:
class DRCD():
  def __init__(self,model_path,model_type): 
    
    # device
    self.device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    
    # model & tokenizer
   # model_type = 'hfl/chinese-roberta-wwm-ext'
    config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
    self.tokenizer = BertTokenizer.from_pretrained(model_type)
    self.model = bertDRCD(model_type).to(self.device)
    self.model.load_state_dict(torch.load(model_path)) 
  
    # 繁簡轉換
    #self.c2tw = OpenCC('s2t') # china to tw
    #self.tw2c = OpenCC('t2s') # tw to china

  def process(self,content,question):
    with torch.no_grad():
      
      token_tensor = self.tokenizer.encode_plus(str(question),str(content),max_length=512,truncation=True,pad_to_max_length=True)
      
      token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(self.device)
      segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(self.device)
      mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(self.device)
      answer_start,answer_end = self.model(input_ids=token,attention_mask=mask,token_type_ids=segment) 
      
      # get ans
      
      tokens = self.tokenizer.convert_ids_to_tokens(token.squeeze())
      

      answer_start = answer_start.argmax(1)
      answer_end = answer_end.argmax(1)
      print(answer_start,answer_end)
      answer = ' '.join(tokens[answer_start:answer_end+1])
      #print('Answer: "' + answer + '"')
    return answer

In [ ]:
d = DRCD(f'saved_models/{modelname}',"bert-base-chinese")

In [ ]:

#content = '福茂唱片音樂股份有限公司是一家臺灣唱片公司，成立於1961年，由福茂工程創辦人張人鳳及次子張耕宇先生創立。曾為環球音樂旗下子公司之一。至2002年，正式退出環球音樂旗下，成為獨立唱片公司。成立初期，福茂為英國迪卡唱片在台灣的獨家代理公司，1986年成立國語部，開始發展國語唱片市場，庾澄慶為第一個與福茂唱片簽約的台灣歌手。其後福茂一手捧紅歌手包括庾澄慶、邰正宵、辛曉琪、林隆璇和范曉萱等人。1989年福茂唱片引進CD技術，發行的首張CD專輯是庾澄慶的《讓我一次愛個夠》，銷量40萬張，並入選台灣百佳唱片第48位。1992年，迪卡唱片母公司寶麗金唱片向福茂買入60%股權，福茂也同時加入寶麗金唱片旗下；福茂英文名稱因此改為「Decca Records Taiwan Ltd.」並使用迪卡唱片商標。首張以迪卡唱片商標發行的唱片是庾澄慶首張英文專輯《哈林音樂電台》。在此時期，福茂一手捧紅大陸歌手那英和香港歌手周慧敏、蘇永康、王菲等在台灣的國語市場。1999年，寶麗金唱片集團被現時的環球唱片收購，福茂也曾為環球旗下子公司之一。'
content = '2014年世界盃外圍賽，韓國在首輪分組賽以首名出線次輪分組賽，與伊朗、卡達、烏茲別克以及黎巴嫩爭逐兩個直接出線決賽周資格，最後韓國僅以較佳的得失球差壓倒烏茲別克，以小組次名取得2014年世界盃決賽周參賽資格，也是韓國連續八次晉身世界盃決賽周。可惜南韓在決賽周表現不濟，三戰一和兩負小組末席出局。2018年世界盃外圍賽，韓國再次在首輪分組賽以首名出線次輪分組賽，再與伊朗、卡達、烏茲別克同組，同組還有中國及敘利亞。最後韓國以兩分壓倒敘利亞及烏茲別克，再以小組次名取得2018年世界盃決賽周參賽資格，也是韓國連續九次晉身世界盃決賽周。韓國的世界盃成績雖然是亞洲最佳，但在亞洲盃足球賽成績就遠不如世界盃。韓國除了在首兩屆亞洲杯奪冠外，但之後一直與亞洲盃錦標無緣，自1992年至2011年更連續六屆未能打入過亞洲盃決賽。2015年亞洲盃足球賽，韓國以五連勝一球不失的姿態，廿七年來首次打入亞洲盃決賽，對手是東道主澳洲。雖然韓國在分組初賽曾以1-0擊敗澳洲，但這場決賽韓國卻先失一球，最後在下半場補時階段扳平，令比賽進入加時階段，可惜澳洲最後在加時階段攻入致勝一球，最後韓國以1-2敗陣，只得亞軍。'
question = "哪一個國家負責舉辦2015年亞洲盃足球賽?"
e = d.process(content,question)
e